In [1]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

In [3]:
def remove_urls(text):
    return re.sub(r'http\S+|www.\S+', '', text)

In [4]:
def find_hashtags(text):
    return [word for word in text.split() if word.startswith('#')]

In [5]:
def remove_hashtags(text):
    return re.sub(r'#\w+', '', text)

In [6]:
def remove_code(text):
    return ''.join(char for char in text if ord(char) < 128)

In [7]:
def remove_number(text):
    return text.apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

In [8]:
def remove_newlines(text):
    return text.replace('\n', ' ').replace('\t', ' ')

In [9]:
def remove_punctuation(text):
    return (text.str.split(rf'[{string.punctuation}]').str.join(' '))

In [10]:
def string_replace(text):
    return(text)

In [11]:
train_data = pd.read_csv('data/twitter/Tweets_train.csv',encoding = 'latin')
test_data = pd.read_csv('data/twitter/Tweets_test.csv',encoding = 'latin')

with open('data/twitter/words.txt') as word_file:
        Word = set(word_file.read().split())

In [12]:
train_data = train_data.drop(['tweet_id'], axis=1)
test_data = test_data.drop(['tweet_id'], axis=1)

In [13]:
train_data['text'] = [remove_mentions(text) for text in train_data['text']]
test_data['text'] = [remove_mentions(text) for text in test_data['text']]

In [14]:
train_data['hashtags'] = train_data['text'].apply(find_hashtags)
test_data['hashtags'] = test_data['text'].apply(find_hashtags)

In [15]:
train_data['text'] =[remove_hashtags(text) for text in train_data['text']]
test_data['text'] =[remove_hashtags(text) for text in test_data['text']]

In [16]:
train_data['text'] =[remove_urls(text) for text in train_data['text']]
test_data['text'] =[remove_urls(text) for text in test_data['text']]

In [17]:
train_data['text'] =[remove_code(text) for text in train_data['text']]
test_data['text'] =[remove_code(text) for text in test_data['text']]

In [18]:
train_data['text'] =[remove_newlines(text) for text in train_data['text']]
test_data['text'] =[remove_newlines(text) for text in test_data['text']]

In [19]:
train_data['text'] = remove_punctuation(train_data['text'])
test_data['text'] = remove_punctuation(test_data['text']) 

In [20]:
train_data['text'] = remove_number(train_data['text'])
test_data['text'] = remove_number(test_data['text']) 

In [21]:
train_data['airline_sentiment'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [22]:
le = LabelEncoder()
train_data['airline_sentiment'] = le.fit_transform(train_data['airline_sentiment'])
test_data['airline_sentiment'] = le.fit_transform(test_data['airline_sentiment'])

In [23]:
X_train, X_test, y_train, y_test = train_data['text'],test_data['text'],train_data['airline_sentiment'],test_data['airline_sentiment']

In [24]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [25]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [26]:
from sklearn.ensemble import RandomForestClassifier
RFmodel = RandomForestClassifier()
RFmodel.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
y_pred2 = RFmodel.predict(X_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred2)
print("accuracy:", accuracy)

accuracy: 0.7684426229508197
